# Tutorial 2 - case study on Norwegian dishwashers

## Intro

This Jupyter notebook includes the calculations behind a case study on household dishwashers in Norway in the years 1950-2050. 

The notebook is divided into four parts:
1. Imports and basic setup - includes Python imports and basic variable definition;
2. Data preprocessing - includes parameter interpolation and regression, and the calculation of the number of dwellings (first driver behind the stock of dishwashers);
3. Retrospective analysis - includes model calculations for the years 1950-2022, including the calculation of the product ownership per dwelling (the second driver behind the stock of dishwashers) and its extrapolation until 2050;
4. Prospective analysis - includes model calculations for the years 1950-2050;
5. Plotting and exporting - visualization of the results.

More details can be found in the publication:
> Krych, K., Müller, DB. & Pettersen, JB. (2024). The ‘nature’ and ‘nurture’ of product lifetimes in dynamic stock modeling. Journal of Industrial Ecology. 

## 1. Imports and basic setup

In [ ]:
import sys, os
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import scipy.stats
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
sys.path.append(os.getcwd() + '/..')
import dynamic_lifetime_model as dlm

In [ ]:
durable = 'dishwasher'
MyYears_retro = list(range(1950, 2022+1))
MyYears_full = list(range(1950, 2050+1))
excel = 'tutorial2_data.xlsx'
overwrite = False
export_figs_to_pdf = True
export_data_to_xlsx = True

In [ ]:
# function used to overlay data in preexisting Excel files
def df_to_excel_overlay(excel, df, sheet_name):
    writer = pd.ExcelWriter(excel, mode='a', if_sheet_exists="overlay", engine='openpyxl') 
    df.to_excel(writer, sheet_name=sheet_name, index=False,startrow=1, header=False)
    writer.close()

# function used to save data in Excel files without deleting other sheets (for Supplementary Information)
def df_to_excel_SI(excel, df, sheet_name):
    writer = pd.ExcelWriter(excel, mode='a', if_sheet_exists="overlay", engine='openpyxl') 
    df.to_excel(writer, sheet_name=sheet_name, index=True,header=True)
    writer.close()

excel_SI = 'SI_The_nature_and_nurture_of_product_lifetimes.xlsx'

## 2. Data preprocessing

Data preprocessing is requires as multiple of the parameters used in the study are not full time series, thus interpolation or regression is needed. Using the obtained time series, we calculate the number of dwellings, which is one of the two drivers for the number of dishwashers in the system (the other being product ownership per dwelling - POpD). 

### Inflows of dishwashers (I) - interpolation

In [ ]:
def perform_inflows_interpolation(df_d, MyYears):
    min_year = min(df_d['time']) # the first year with data
    max_year = max(df_d['time']) # the last year with data
    if min_year > MyYears[0]:
        df_d = pd.concat([df_d, pd.DataFrame({"time": [MyYears[0]], "value": [0]})], ignore_index=True)
        df_d = pd.concat([df_d, pd.DataFrame({"time": [min_year-1], "value": [0]})], ignore_index=True)
    if max_year < MyYears[-1]:
        last_value = df_d[df_d['time']==max_year]['value'].item()
        df_d = pd.concat([df_d, pd.DataFrame({"time": [MyYears[-1]], "value": [last_value]})], ignore_index=True)
    df_d = df_d.groupby('time').mean(numeric_only=True).reset_index()
    x_data = df_d['time']
    y_data = df_d['value']
    f_linear = interp1d(x_data, y_data)
    return f_linear(MyYears)

In [ ]:
df_i_data = pd.read_excel(excel, sheet_name='I_data')
df_i_ip = pd.DataFrame(data=perform_inflows_interpolation(df_i_data, MyYears_retro),
                       index=pd.MultiIndex.from_product([MyYears_retro], names=['time']), columns=['value'])
df_i_ip = df_i_ip.reset_index()

In [ ]:
if overwrite:
    df_to_excel_overlay(excel,df_i_ip,'I')

### People per dwelling (PpD) - regression

In [ ]:
def logistic(x, ti, a, C0, C1):
    """
    ti: inflection time
    a: slope
    C0: start value
    C1: end value
    x: vector of observation points (time)
    """
    return (C1 - C0) / (1 + np.exp(-a * (x - ti))) + C0

In [ ]:
df_ppd_data = pd.read_excel(excel, sheet_name='PpD_data')
lower_bounds = [1900, 0, 5, 0]
higher_bounds = [2100, 1, 10, 10]
popt, pcov = curve_fit(logistic, df_ppd_data['time'], df_ppd_data['value'], bounds=[lower_bounds, higher_bounds])
df_ppd_rg = pd.DataFrame(data=logistic(MyYears_full,*popt), index=pd.MultiIndex.from_product([MyYears_full], names=['time']), columns=['value'])
df_ppd_rg = df_ppd_rg.reset_index()

In [ ]:
if overwrite:
    df_to_excel_overlay(excel,df_ppd_rg,'PpD')

### Cabins per person (CpP) - regression

In [ ]:
def asymmetric_logistic(x, ti, a, C0, C1, v):
    """
    ti: inflection time
    a: slope
    C0: start value
    C1: end value
    x: vector of observation points (time)
    v: assymetry factor
    """
    return (C1 - C0) / (1 + np.exp(-a * (x - ti))) ** (1 / v) + C0

In [ ]:
df_cpp_data = pd.read_excel(excel, sheet_name='CpP_data')
CpP_years = list(range(1900,2050+1))
lower_bounds =  [1950, 0, 0,      0, 0]
higher_bounds = [1990, 1, 0.0001, 0.12, 10]
popt, pcov = curve_fit(asymmetric_logistic, df_cpp_data['time'], df_cpp_data['value'], bounds=[lower_bounds, higher_bounds])
df_cpp_rg = pd.DataFrame(data=asymmetric_logistic(CpP_years,*popt), 
                         index=pd.MultiIndex.from_product([CpP_years], names=['time']), columns=['value'])
df_cpp_rg = df_cpp_rg.reset_index()
print(popt)
plt.plot(df_cpp_rg['time'], df_cpp_rg['value'])
plt.scatter(df_cpp_data['time'], df_cpp_data['value'])

In [ ]:
if overwrite:
    df_to_excel_overlay(excel,df_cpp_rg,'CpP')

### Share of cabins electrified (SoCE) - regression

In [ ]:
df_soce = pd.read_excel(excel, sheet_name='SoCE_data', usecols="A:D")
def linear(x, a, b):
    return a * x + b
x_data, y_data = df_soce['time'], df_soce['value']
with np.errstate(divide='ignore'): # to ignore "RuntimeWarning: divide by zero encountered in..."
    y_data_log = np.log(y_data*100)
y_data_log[y_data_log == -np.inf] = 0
popt, pcov = curve_fit(linear, x_data, y_data_log)
soce = np.exp(popt[0]*np.array(MyYears_full)+popt[1])/100
soce[soce >1] = 1
soce[:1960-min(df_soce['time'])] = 0 
soce[soce <0] = 0
df_soce_rg = pd.DataFrame(data=soce, index=pd.MultiIndex.from_product([MyYears_full], names=['time']), columns=['value'])
df_soce_rg = df_soce_rg.reset_index()

In [ ]:
if overwrite:
    df_to_excel_overlay(excel,df_soce_rg,'SoCE')

### The number of dwellings (D)

Once we have all the required time series, the number of dwellings can be calculated. It is a sum of primary dwellings (calculated using the population P and the people per dwelling PpD) and secondary dwellings that are equipped with dishwashers (calculated using population P, cabins per person CpP and share of cabins electrified SoCE). 

In [ ]:
df_ppd = pd.read_excel(excel, sheet_name='PpD')
df_p = pd.read_excel(excel, sheet_name='P')
df_cpp = pd.read_excel(excel, sheet_name='CpP')
df_soce = pd.read_excel(excel, sheet_name='SoCE')
df_k_cab = pd.read_excel(excel, sheet_name='k-cab')
df_l_cab = pd.read_excel(excel, sheet_name='lambda-cab')

In [ ]:
df_p_short = df_p[df_p['time']>=1950]
df_p_short = df_p_short.reset_index()
dwellings = np.array(df_p_short['value']/df_ppd['value'])

t = df_cpp['time']
s = df_cpp['value']*df_p['value']
scale = df_l_cab['value'].values[0]
shape = df_k_cab['value'].values[0]
sf = np.zeros((len(t), len(t)))
for m in range(0, len(t)):  # cohort index
    sf[m::,m] = scipy.stats.weibull_min.sf(np.arange(0,len(t)-m), c=shape, loc = 0, scale=scale)

# MFA calculations start (assuming sf[0] != 0 and no negative inflows)
i = np.zeros(len(t))
s_c = np.zeros((len(t), len(t)))
i[0] = s[0] / sf[0, 0]
s_c[:, 0] = i[0] * sf[:, 0]
for m in range(1, len(t)):
    i[m] = (s[m] - s_c[m, :].sum()) / sf[m,m]
    s_c[m::, m] = i[m] * sf[m::, m]

o_c = np.zeros_like(s_c)
o_c[1::,:] = -1 * np.diff(s_c,n=1,axis=0)
o_c[np.diag_indices(len(t))] = i - np.diag(s_c) # allow for outflow in year 0 already

soce = df_soce['value'] # share of cabins electrified
soce[soce >1] = 1
soce[:1960-min(df_soce['time'])] = 0 
soce[soce <0] = 0
el_cabins = np.einsum('tc,c->t',s_c[50:,50:],soce)
all_dwellings = dwellings+el_cabins

In [ ]:
df_d = pd.DataFrame(data=all_dwellings, index=pd.MultiIndex.from_product([MyYears_full], names=['time']), columns=['value'])
df_d = df_d.reset_index()
df_d['unit'] = 'dwellings'
df_d['source'] = 'calculated using a dwelling sub-model'


In [ ]:
if overwrite:
    df_to_excel_overlay(excel,df_d,'D')

## 3. Retrospective analysis

The retrospective analysis runs during years 1950-2022 and is an inflow-driven model used to calculate the stock of dishwashers. The stock then serves to calculate product ownership per dwelling (POpD) using the number of dwellings calculated above. Then, the ownership per dwelling is extrapolated until 2050, assuming saturation at 1.05 dishwashers/dwelling.

In [ ]:
i_retro = pd.read_excel(excel, sheet_name='I')['value'].values
shape = pd.read_excel(excel, sheet_name='k')['value'].values
scale = pd.read_excel(excel, sheet_name='lambda')['value'].values

In [ ]:
model_retro = dlm.DynamicLifetimeModel(i=i_retro, t=MyYears_retro)
model_retro.lt = {'Type': 'Weibull', 
          'Scale': model_retro.create_2Darray(scale), #two-dimensional parameter (time by cohort)
          'Shape': model_retro.create_2Darray(shape)  #two-dimensional parameter (time by cohort)
          }
model_retro.compute_inflow_driven_model()

### Calculate product ownership per dwelling (POpD)

In [ ]:
df_d = pd.read_excel(excel, sheet_name='D')
df_d_retro = df_d[df_d['time']<=2022]
d_retro = df_d_retro['value'].values
d = df_d['value'].values

In [ ]:
popd_retro = model_retro.s/d_retro

In [ ]:
df_popd_data = pd.read_excel(excel, sheet_name='POpD_data')
plt.scatter(df_popd_data['time'], df_popd_data['value'])
plt.plot(MyYears_retro, popd_retro)

### Extrapolate POpD

In [ ]:
popd_max = 1.05
def logistic2(x, ti, a, C, C1):
    C0 = C-(C1-C)
    return (C1 - C0) / (1 + np.exp(-a * (x - ti))) + C0
slope = (popd_retro[-1]-popd_retro[-5])/(MyYears_retro[-1]-MyYears_retro[-5])*popd_max/(popd_retro[-1]*(popd_max-popd_retro[-1]))
popd_full = [x for x in popd_retro[:-1]] + list(logistic2(np.arange(2019,2050-3+1),2019,slope,popd_retro[-1],popd_max))
plt.plot(MyYears_retro, popd_retro, label='historical')
plt.plot(MyYears_full[2022-1950:], popd_full[2022-1950:], label='extrapolated')
plt.axvline(x=2022,color="grey", linestyle="--")
plt.legend()
plt.show()

In [ ]:
df_popd = pd.DataFrame(data=popd_full, index=pd.MultiIndex.from_product([MyYears_full], names=['time']), columns=['value'])
df_popd = df_popd.reset_index()
df_popd['unit'] = 'dishwashers/dwelling'
df_popd['source'] = 'retrospective model and extrapolation'

In [ ]:
if overwrite:
    df_to_excel_overlay(excel,df_popd,'POpD')

## 4. Prospective analysis

Using the parameters derived in the previous stages of this tutorial, it is possible to run a stock-driven analysis for the years 1950-2050. Five models are created, including a baseline model and four lifetime extension models. 

In [ ]:
t = MyYears_full
popd = pd.read_excel(excel, sheet_name='POpD')['value'].values
d = pd.read_excel(excel, sheet_name='D')['value'].values
stock = popd*d
shape = pd.read_excel(excel, sheet_name='k')['value'].values
scale = pd.read_excel(excel, sheet_name='lambda')['value'].values
lt_ext =  0.2

### Calculate the scenarios

In [ ]:
model_0 = dlm.DynamicLifetimeModel(s=stock, t=t)
model_0.lt = {'Type': 'Weibull', 
          'Scale': model_0.create_2Darray(scale),
          'Shape': model_0.create_2Darray(shape)
          }
model_0.compute_stock_driven_model()

In [ ]:
# lifespan labelling (cohort effect)
model_1 = dlm.DynamicLifetimeModel(s=stock, t=t)
model_1.lt = {'Type': 'Weibull', 
                'Scale': model_1.create_2Darray(scale), 
                'Shape': model_1.create_2Darray(shape)
                }
model_1.lt['Scale'] = model_1.add_cohort_effect(model_1.lt['Scale'],1+lt_ext, 2026, ref='relative')
model_1.compute_stock_driven_model()

In [ ]:
# tax relief/subsidy (period effect)
model_2 = dlm.DynamicLifetimeModel(s=stock, t=t)
model_2.lt = {'Type': 'Weibull', 
                'Scale': model_2.create_2Darray(scale), 
                'Shape': model_2.create_2Darray(shape)
                }
model_2.lt['Scale'] = model_2.add_period_effect(model_2.lt['Scale'],1+lt_ext,2035, ref='relative', trans_start=2026, trans_type='logistic')
model_2.compute_stock_driven_model()

In [ ]:
# affordable spare parts (age-cohort effect)
model_3 = dlm.DynamicLifetimeModel(s=stock, t=t)
model_3.lt = {'Type': 'Weibull', 
                'Scale': model_3.create_2Darray(scale), 
                'Shape': model_3.create_2Darray(shape)
                }
model_3.lt['Scale'] = model_1.add_cohort_effect(model_3.lt['Scale'],1+lt_ext,2035, ref='relative', ages=np.arange(10), trans_start=2026, trans_type='logistic')
model_3.compute_stock_driven_model()

In [ ]:
# guarranty (age-period effect)
model_4 = dlm.DynamicLifetimeModel(s=stock, t=t)
model_4.lt = {'Type': 'Weibull', 
                'Scale': model_4.create_2Darray(scale), 
                'Shape': model_4.create_2Darray(shape)
                }
model_4.lt['Scale'] = model_4.add_period_effect(model_4.lt['Scale'],1+lt_ext,2035, ref='relative', ages=np.arange(5), trans_start=2026, trans_type='logistic')
model_4.compute_stock_driven_model()

## 5. Plotting and exporting

The calculations are successful, therefore we can plot the results and export them if needed.

### Results - inflows

In [ ]:
import seaborn as sns
palette = sns.color_palette("rocket", 5)

In [ ]:
fig = plt.figure(figsize=(6, 3))
fig.set_dpi(800)
plt.plot(t[:76],model_0.i[:76], label='Historical data', color='gray', linewidth=2)
plt.plot(t[75:],model_0.i[75:], label='Scenario 0: baseline', linewidth=2, color=palette[0])
plt.plot(t[75:],model_1.i[75:], label='Scenario 1: lifespan labelling', linewidth=2, linestyle=(0, (1, 1)), color=palette[1])
plt.plot(t[75:],model_2.i[75:], label='Scenario 2: repair subsidies', linewidth=2, linestyle='--', color=palette[2])
plt.plot(t[75:],model_3.i[75:], label='Scenario 3: spare parts', linewidth=2, linestyle='-.', color=palette[3])
plt.plot(t[75:],model_4.i[75:], label='Scenario 4: guarantee', linewidth=2, linestyle=':', color=palette[4])
plt.axvline(2025, color='gray', linestyle='--')
fig.legend(fontsize=9, bbox_to_anchor=(1,0.5),loc="center left", handlelength=4)
plt.xlim(1990-2,2050+2)
plt.ylim(0,)
plt.xlabel('Time')
plt.ylabel('Sales of new dishwashers')
plt.tight_layout()
plt.show()
if export_figs_to_pdf:
    fig.savefig('Fig5.pdf', format='pdf', bbox_inches = "tight")
if export_data_to_xlsx:
    data = np.concatenate(([model_0.i],[model_1.i],[model_2.i],[model_3.i],[model_4.i]), axis=0)
    col_names = ['baseline', 'scenario 1', 'scenario 2', 'scenario 3', 'scenario 4']
    df = pd.DataFrame(data=data.T, index=pd.MultiIndex.from_product([MyYears_full], names=['time']), columns=col_names)
    df_to_excel_SI(excel_SI,df,'Figure 5')

In [ ]:
print(f"Sales decrease in 2050 relative to baseline:" 
      f"\nScenario 1: {round((model_1.i[-1]-model_0.i[-1])/model_0.i[-1]*100,1)}%"
      f"\nScenario 2: {round((model_2.i[-1]-model_0.i[-1])/model_0.i[-1]*100,1)}%"
      f"\nScenario 3: {round((model_3.i[-1]-model_0.i[-1])/model_0.i[-1]*100,1)}%"
      f"\nScenario 4: {round((model_4.i[-1]-model_0.i[-1])/model_0.i[-1]*100,1)}%")

### Results - age of stock and outflows

In [ ]:
fig = plt.figure(figsize=(10, 4))
gs = fig.add_gridspec(1, 2,wspace=0.2)
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(t[:76],model_0.calculate_age_stock(scale_by_inflow=True)[:76], color='gray', linewidth=2)
ax1.plot(t[75:],model_0.calculate_age_stock(scale_by_inflow=True)[75:], linewidth=2, color=palette[0])
ax1.plot(t[75:],model_1.calculate_age_stock(scale_by_inflow=True)[75:], linewidth=2, linestyle=(0, (1, 1)), color=palette[1])
ax1.plot(t[75:],model_2.calculate_age_stock(scale_by_inflow=True)[75:], linewidth=2, linestyle='--', color=palette[2])
ax1.plot(t[75:],model_3.calculate_age_stock(scale_by_inflow=True)[75:], linewidth=2, linestyle='-.', color=palette[3])
ax1.plot(t[75:],model_4.calculate_age_stock(scale_by_inflow=True)[75:], linewidth=2, linestyle=':', color=palette[4])
ax1.axvline(2025, color='gray', linestyle='--')
ax1.set_xlim(2000,2050)
ax1.set_ylim(0,18)
ax1.set_title('Age of stock')

ax2 = fig.add_subplot(gs[0, 1])
ax2.plot(t[:76],model_0.calculate_age_outflow(scale_by_inflow=True)[:76], label='Historical data', color='gray', linewidth=2)
ax2.plot(t[75:],model_0.calculate_age_outflow(scale_by_inflow=True)[75:], label='Scenario 0: baseline', linewidth=2, color=palette[0])
ax2.plot(t[75:],model_1.calculate_age_outflow(scale_by_inflow=True)[75:], label='Scenario 1: lifespan labelling', linewidth=2, linestyle=(0, (1, 1)), color=palette[1])
ax2.plot(t[75:],model_2.calculate_age_outflow(scale_by_inflow=True)[75:], label='Scenario 2: repair subsidies', linewidth=2, linestyle='--', color=palette[2])
ax2.plot(t[75:],model_3.calculate_age_outflow(scale_by_inflow=True)[75:], label='Scenario 3: spare parts', linewidth=2, linestyle='-.', color=palette[3])
ax2.plot(t[75:],model_4.calculate_age_outflow(scale_by_inflow=True)[75:], label='Scenario 4: guarantee', linewidth=2, linestyle=':', color=palette[4])
ax2.axvline(2025, color='gray', linestyle='--')
ax2.set_xlim(2000,2050)
ax2.set_ylim(0,18)
ax2.set_title('Age of outflows')

fig.legend(bbox_to_anchor=(0.5,-0.02), loc="upper center", fontsize=10)
plt.show()

### Results - hazard matrix

In [ ]:
def plot_matrix_heatmap(matrix, ax=None, colorbar=True):
    if ax == None:
        plt.figure(figsize=(4, 3))
        ax = plt.gca()
    mask = np.zeros_like(matrix)
    mask[np.triu_indices_from(mask)] = True
    plot_matrix_mask = np.ma.masked_array(matrix, mask=mask)
    img = ax.imshow(plot_matrix_mask,cmap='Oranges')
    ax.set_xticks(ticks=np.arange(0, t[-1]-t[40]+1, 15), labels=t[40::15], rotation=90)
    ax.set_yticks(ticks=np.arange(0, t[-1]-t[40]+1, 15), labels=t[40::15])
    if colorbar:
        cbar = plt.colorbar(img, ax=ax)
        # cbar = plt.colorbar(img,cax=fig.add_axes([0.98, 0.11, 0.02, 0.77]))
        cbar.ax.set_ylabel('Hazard rate')
    return

In [ ]:
fig = plt.figure(figsize=(12, 6))
gs = fig.add_gridspec(2, 3,wspace=0.5, hspace=0.5)
ax0 = fig.add_subplot(gs[0, 0])
plot_matrix_heatmap(model_0.hz[40:, 40:], ax=ax0, colorbar=False)
ax0.set_title('Scenario 0: baseline')
ax1 = fig.add_subplot(gs[0, 1])
plot_matrix_heatmap(model_1.hz[40:, 40:], ax=ax1, colorbar=False)
ax1.set_title('Scenario 1: ecodesign and labelling')
ax2 = fig.add_subplot(gs[0, 2])
plot_matrix_heatmap(model_2.hz[40:, 40:], ax=ax2, colorbar=False)
ax2.set_title('Scenario 2: repair subsidies')
ax3 = fig.add_subplot(gs[1, 1])
plot_matrix_heatmap(model_3.hz[40:, 40:], ax=ax3, colorbar=False)
ax3.set_title('Scenario 3: spare parts')
ax4 = fig.add_subplot(gs[1, 2])
plot_matrix_heatmap(model_4.hz[40:, 40:], ax=ax4, colorbar=False)
ax4.set_title('Scenario 4: warranties')

fig.suptitle('Hazard matrix', fontsize=20)
fig.subplots_adjust(top=0.88)
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 6))
gs = fig.add_gridspec(2, 3,wspace=0.5, hspace=0.5)
ax0 = fig.add_subplot(gs[0, 0])
plot_matrix_heatmap(model_0.hz[40:, 40:]-model_0.hz[40:, 40:], ax=ax0, colorbar=False)
ax0.set_title('Scenario 0: baseline')
ax1 = fig.add_subplot(gs[0, 1])
plot_matrix_heatmap(model_0.hz[40:, 40:]-model_1.hz[40:, 40:], ax=ax1, colorbar=False)
ax1.set_title('Scenario 1: ecodesign and labelling')
ax2 = fig.add_subplot(gs[0, 2])
plot_matrix_heatmap(model_0.hz[40:, 40:]-model_2.hz[40:, 40:], ax=ax2, colorbar=False)
ax2.set_title('Scenario 2: repair subsidies')

ax3 = fig.add_subplot(gs[1, 1])
plot_matrix_heatmap(model_0.hz[40:, 40:]-model_3.hz[40:, 40:], ax=ax3, colorbar=False)
ax3.set_title('Scenario 3: spare parts')
ax4 = fig.add_subplot(gs[1, 2])
plot_matrix_heatmap(model_0.hz[40:, 40:]-model_4.hz[40:, 40:], ax=ax4, colorbar=False)
ax4.set_title('Scenario 4: warranties')

plt.suptitle('Hazard matrix (baseline minus given scenario)', fontsize=16)
fig.subplots_adjust(top=0.88)
plt.tight_layout()
plt.show()
